In [9]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import VideoProcessor
import AudioProcessor

In [10]:
db_conn = db.get_connection()
video_id = "7442757197278366998"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [11]:
video_url, video_path, audio_path = download.tiktok_full(video_id)


INFO - [EnrichmentWorker] - Downloading MP4 for 7442757197278366998 → /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7442757197278366998.mp4


KeyboardInterrupt: 

In [12]:
model_name = "ggml-large-v3.bin" 
transcript, lang, output = transcribe_audio(audio_path, model_name)
logging.info(transcript)

INFO - [EnrichmentWorker] - Starting transcriber model ggml-large-v3.bin for audio in /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7442757197278366998.wav
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO - [EnrichmentWorker] - The truth is plain to see.
 If you want freedom, take pride in your country.
 If you want democracy, hold on to your sovereignty.
 And if you want peace, love your nation.
 Wise leaders always put the good of their own people
 and their own country first.
 The future does not belong to globalists.
 The future belongs to patriots.
 The future belongs to sovereign and independent nations
 who protect their citizens, respect their neighbors,
 and honor the differe

In [14]:
sentiment_results = SentimentAnalyzer.get_sentiment(transcript, "ro")
sentiment_results

INFO - [EnrichmentWorker] - SENTIMENT MODEL OUTPUT SequenceClassifierOutput(loss=None, logits=tensor([[0.1536, 0.3278]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
INFO - [EnrichmentWorker] - SENTIMENT MODEL SCORES [0.4565397 0.5434603]


{'positive': 0.5434603095054626,
 'neutral': 0.0,
 'negative': 0.45653969049453735}

In [15]:
description = get_video_description(video_id, db_conn)
description

'𝐕𝐢𝐢𝐭𝐨𝐫𝐮𝐥 𝐍𝐚ț𝐢𝐮𝐧𝐢𝐥𝐨𝐫 𝐒𝐮𝐯𝐞𝐫𝐚𝐧𝐞: 𝐌𝐞𝐬𝐚𝐣𝐮𝐥 𝐥𝐮𝐢 𝐃𝐨𝐧𝐚𝐥𝐝 𝐓𝐫𝐮𝐦𝐩 𝐝𝐞𝐬𝐩𝐫𝐞 𝐂ă𝐥𝐢𝐧 𝐆𝐞𝐨𝐫𝐠𝐞𝐬𝐜𝐮 #DonaldTrump #CălinGeorgescu #Suveranitate #Libertate #Democrație #Patriotism #Pace #NațiuniSuverane #LideriÎnțelepți #BinelePoporului #ViitorulPatrioților #MândrieNațională #ProtecțiaCetățenilor #RespectNațiuni #Independență '

In [ ]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)

In [ ]:
analysis_result_json